In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (4701 Meridian Ave #221, Miami Beach, FL 33140...
2     (1201 20th St Apt 302, Miami Beach, FL 33139, ...
3     (100 South Pointe Dr APT 1403, Miami Beach, FL...
4     (20165 NE 39th Pl APT 401, Aventura, FL 33180,...
5     (2 Grove Isle Dr B902, Miami, FL 33133, USA, (...
6     (801 S Miami Ave Unit 3001, Miami, FL 33130, U...
7     (330 Sunny Isles Blvd, Sunny Isles Beach, FL 3...
8     (529 SW 11th St #104, Miami, FL 33129, USA, (2...
9     (141 Crandon Blvd APT 240, Key Biscayne, FL 33...
10    (1200 Brickell Bay Dr APT 3901, Miami, FL 3313...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,The Ritz Carlton Residences\r4701 N Meridian 2...,11/9/2022,942,"$5,000,000.00",3265,"$1,531.39",Juan Carlos Pagan,HAUS Realty,JTHS Accounting,MIAMI Association of REALTORS®,The Ritz Carlton Residences 4701 N Meridian 22...,The Ritz Carlton Residences,4701 N Meridian 221 Miami Beach,"(4701 Meridian Ave #221, Miami Beach, FL 33140...","(25.8230945, -80.1310154, 0.0)",25.823094,-80.131015,0.0
2,Palau Sunset Harbour\r1201 20th St 302\rMiami ...,11/7/2022,101,"$2,600,000.00",1728,"$1,504.63",Menachem Fellig,"Compass Florida, LLC.",John Sandberg,Douglas Elliman,Palau Sunset Harbour 1201 20th St 302 Miami Beach,Palau Sunset Harbour,1201 20th St 302 Miami Beach,"(1201 20th St Apt 302, Miami Beach, FL 33139, ...","(25.795997, -80.14206039999999, 0.0)",25.795997,-80.142060,0.0
3,Continuum South Tower\r100 S Pointe Dr 1403\rM...,11/7/2022,151,"$2,525,000.00",1365,"$1,849.82",Adam Amsterdam,LoKation,John Lennon,South Pointe Drive Realty Inc,Continuum South Tower 100 S Pointe Dr 1403 Mia...,Continuum South Tower,100 S Pointe Dr 1403 Miami Beach,"(100 South Pointe Dr APT 1403, Miami Beach, FL...","(25.7676323, -80.137348, 0.0)",25.767632,-80.137348,0.0
4,Porto Vita\r20165 NE 39th Pl 401\rAventura,11/9/2022,89,"$2,300,000.00",3800,$605.26,Sylvia Fragos,Great Estates Int'l Realty,Stuart Berger,Coldwell Banker Realty,Porto Vita 20165 NE 39th Pl 401 Aventura,Porto Vita,20165 NE 39th Pl 401 Aventura,"(20165 NE 39th Pl APT 401, Aventura, FL 33180,...","(25.9625045, -80.12481729999999, 0.0)",25.962505,-80.124817,0.0
5,Grove Isle Condo\r2 Grove Isle Dr B902\rMiami,11/9/2022,78,"$1,500,000.00",2398,$625.52,David Sporn,Coldwell Banker Realty,Bethea Goldman,BHHS EWM Realty,Grove Isle Condo 2 Grove Isle Dr B902 Miami,Grove Isle Condo,2 Grove Isle Dr B902 Miami,"(2 Grove Isle Dr B902, Miami, FL 33133, USA, (...","(25.7351295, -80.21874609999999, 0.0)",25.735129,-80.218746,0.0
6,SLS Lux Brickell Residence\r801 S Miami Ave 30...,11/7/2022,8,"$1,400,000.00",1613,$867.95,Karen Vieira Ferreira,Miami Realty Solution Group,Mariela Feo La Cruz,Realty One Group Evolution,SLS Lux Brickell Residence 801 S Miami Ave 300...,SLS Lux Brickell Residence,801 S Miami Ave 3001 Miami,"(801 S Miami Ave Unit 3001, Miami, FL 33130, U...","(25.7660758, -80.1929055, 0.0)",25.766076,-80.192905,0.0
7,Parque Tower\r330 Sunny Isles Blvd Water View\...,11/8/2022,38,"$1,100,000.00",1524,$721.78,Heloisa Arazi,AMG International Realty,Patricia Hernandez,AMG International Realty,Parque Tower 330 Sunny Isles Blvd Water View S...,Parque Tower,330 Sunny Isles Blvd Water View Sunny Isles Beach,"(330 Sunny Isles Blvd, Sunny Isles Beach, FL 3...","(25.9294643, -80.1259521, 0.0)",25.929464,-80.125952,0.0
8,Eleven in the Roads\r529 SW 11th St 104\rMiami,11/7/2022,55,"$1,012,500.00",1527,$663.06,Diego Sequera,Edivial International Realty,Alexandra Sierra,"Compass Florida, LLC.",Eleven in the Roads 529 SW 11th St 104 Miami,Eleven in the Roads,529 SW 11th St 104 Miami,"(529 SW 11th St #104, Miami, FL 33129, USA, (2...","(25.763518, -80.202933, 0.0)",25.763518,-80.202933,0.0
9,Key Colony No 4 Condo\r141 Crandon Blvd 240\rK...,11/8/2022,32,"$1,000,000.00",1715,$583.09,Pity Betancourt,BHHS EWM Realty,Pedro Mingo,Unique International Properties,Key Colony No 4 Condo 141 Crandon Blvd 240 Key...,Key Colony No,4 Condo 141 Crandon Blvd 240 Key Biscayne,"(141 Crandon Blvd APT 240, Key Biscayne, FL 33...","(25.6977458, -80.16075560000002, 0.0)",25.697746,-80.160756,0.0
10,0The Club at Brickell Bay\r1200 Brickell Bay D...,11/7/2022,36,"$1,000,000.00",1105,$904.98,Jessica Bechard,Homeland Capital Realty,Yesenia Garcia,Homeland Capital Realty,0The Club at Brickell Bay 1200 Brickell Bay Dr...,,0The Club at Brickell Bay 1200 Brickell Bay Dr...,"(1200 Brickell Bay Dr APT 3901, Miami, FL 3313...","(25.7618036, -80.19058

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,The Ritz Carlton Residences\r4701 N Meridian 2...,11/9/2022,942,"$5,000,000.00",3265,"$1,531.39",Juan Carlos Pagan,HAUS Realty,JTHS Accounting,MIAMI Association of REALTORS®,The Ritz Carlton Residences 4701 N Meridian 22...,The Ritz Carlton Residences,4701 N Meridian 221 Miami Beach,"(4701 Meridian Ave #221, Miami Beach, FL 33140...","(25.8230945, -80.1310154, 0.0)",25.823094,-80.131015,0.0
2,Palau Sunset Harbour\r1201 20th St 302\rMiami ...,11/7/2022,101,"$2,600,000.00",1728,"$1,504.63",Menachem Fellig,"Compass Florida, LLC.",John Sandberg,Douglas Elliman,Palau Sunset Harbour 1201 20th St 302 Miami Beach,Palau Sunset Harbour,1201 20th St 302 Miami Beach,"(1201 20th St Apt 302, Miami Beach, FL 33139, ...","(25.795997, -80.14206039999999, 0.0)",25.795997,-80.142060,0.0
3,Continuum South Tower\r100 S Pointe Dr 1403\rM...,11/7/2022,151,"$2,525,000.00",1365,"$1,849.82",Adam Amsterdam,LoKation,John Lennon,South Pointe Drive Realty Inc,Continuum South Tower 100 S Pointe Dr 1403 Mia...,Continuum South Tower,100 S Pointe Dr 1403 Miami Beach,"(100 South Pointe Dr APT 1403, Miami Beach, FL...","(25.7676323, -80.137348, 0.0)",25.767632,-80.137348,0.0
4,Porto Vita\r20165 NE 39th Pl 401\rAventura,11/9/2022,89,"$2,300,000.00",3800,$605.26,Sylvia Fragos,Great Estates Int'l Realty,Stuart Berger,Coldwell Banker Realty,Porto Vita 20165 NE 39th Pl 401 Aventura,Porto Vita,20165 NE 39th Pl 401 Aventura,"(20165 NE 39th Pl APT 401, Aventura, FL 33180,...","(25.9625045, -80.12481729999999, 0.0)",25.962505,-80.124817,0.0
5,Grove Isle Condo\r2 Grove Isle Dr B902\rMiami,11/9/2022,78,"$1,500,000.00",2398,$625.52,David Sporn,Coldwell Banker Realty,Bethea Goldman,BHHS EWM Realty,Grove Isle Condo 2 Grove Isle Dr B902 Miami,Grove Isle Condo,2 Grove Isle Dr B902 Miami,"(2 Grove Isle Dr B902, Miami, FL 33133, USA, (...","(25.7351295, -80.21874609999999, 0.0)",25.735129,-80.218746,0.0
6,SLS Lux Brickell Residence\r801 S Miami Ave 30...,11/7/2022,8,"$1,400,000.00",1613,$867.95,Karen Vieira Ferreira,Miami Realty Solution Group,Mariela Feo La Cruz,Realty One Group Evolution,SLS Lux Brickell Residence 801 S Miami Ave 300...,SLS Lux Brickell Residence,801 S Miami Ave 3001 Miami,"(801 S Miami Ave Unit 3001, Miami, FL 33130, U...","(25.7660758, -80.1929055, 0.0)",25.766076,-80.192905,0.0
7,Parque Tower\r330 Sunny Isles Blvd Water View\...,11/8/2022,38,"$1,100,000.00",1524,$721.78,Heloisa Arazi,AMG International Realty,Patricia Hernandez,AMG International Realty,Parque Tower 330 Sunny Isles Blvd Water View S...,Parque Tower,330 Sunny Isles Blvd Water View Sunny Isles Beach,"(330 Sunny Isles Blvd, Sunny Isles Beach, FL 3...","(25.9294643, -80.1259521, 0.0)",25.929464,-80.125952,0.0
8,Eleven in the Roads\r529 SW 11th St 104\rMiami,11/7/2022,55,"$1,012,500.00",1527,$663.06,Diego Sequera,Edivial International Realty,Alexandra Sierra,"Compass Florida, LLC.",Eleven in the Roads 529 SW 11th St 104 Miami,Eleven in the Roads,529 SW 11th St 104 Miami,"(529 SW 11th St #104, Miami, FL 33129, USA, (2...","(25.763518, -80.202933, 0.0)",25.763518,-80.202933,0.0
9,Key Colony No 4 Condo\r141 Crandon Blvd 240\rK...,11/8/2022,32,"$1,000,000.00",1715,$583.09,Pity Betancourt,BHHS EWM Realty,Pedro Mingo,Unique International Properties,Key Colony No 4 Condo 141 Crandon Blvd 240 Key...,Key Colony No,4 Condo 141 Crandon Blvd 240 Key Biscayne,"(141 Crandon Blvd APT 240, Key Biscayne, FL 33...","(25.6977458, -80.16075560000002, 0.0)",25.697746,-80.160756,0.0
10,0The Club at Brickell Bay\r1200 Brickell Bay D...,11/7/2022,36,"$1,000,000.00",1105,$904.98,Jessica Bechard,Homeland Capital Realty,Yesenia Garcia,Homeland Capital Realty,0The Club at Brickell Bay 1200 Brickell Bay Dr...,,0The Club at Brickell Bay 1200 Brickell Bay Dr...,"(1200 Brickell Bay Dr APT 3901, Miami, FL 3313...","(25.7618036, -80.19058

In [11]:
df.at[10,'building_name']=('The Club at Brickell Bay')
df.at[10,'address_only']=('1200 Brickell Bay Drive')

In [12]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [13]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('November 6th - November 12th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [39]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [35]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [36]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_110722
